In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import string
from PIL import Image

In [2]:
data_df = pd.read_csv("D:/Deep Learning Projects/Image captioning - flickr8k/flickr8k/versions/1/captions.txt")

In [3]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40455 entries, 0 to 40454
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   image    40455 non-null  object
 1   caption  40455 non-null  object
dtypes: object(2)
memory usage: 632.2+ KB


In [4]:
data_df.image.nunique()

8091

In [5]:
data_df.rename(columns={'image':'id'}, inplace=True)

In [6]:
current_directory = os.getcwd()
items = os.listdir(current_directory+"/flickr8k/versions/1/images")
img_array = []
id = []
for file_name in items:
    img = Image.open("D:/Deep Learning Projects/Image captioning - flickr8k/flickr8k/versions/1/images/"+file_name)
    pixel_array = np.array(img)
    id.append(file_name)
    img_array.append(pixel_array)

image_df = pd.DataFrame({"id":id, "pixel_values":img_array})
final_df = pd.merge(data_df, image_df, on='id', how='left')

In [7]:
final_df.to_csv("final_consolidated.csv")

In [8]:
import tensorflow as tf
import tensorflow_hub as hub
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input
import cv2
import torch
from torchvision import transforms

In [9]:
vgg16_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False)

In [10]:
vgg16_model.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
model = Sequential()
model.add(vgg16_model)
for layer in model.layers:
   layer.trainable = True
model.add(Flatten())
model.add(Dense(4096,activation='relu'))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │   102,764,544 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 117,479,232 (448.15 MB)

 Trainable params: 117,479,232 (448.15 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
final_df.head()

,id,caption,pixel_values
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...,"[[[58, 56, 77], [115, 131, 130], [106, 123, 11..."
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .,"[[[58, 56, 77], [115, 131, 130], [106, 123, 11..."
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .,"[[[58, 56, 77], [115, 131, 130], [106, 123, 11..."
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...,"[[[58, 56, 77], [115, 131, 130], [106, 123, 11..."
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...,"[[[58, 56, 77], [115, 131, 130], [106, 123, 11..."


In [13]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

def clean_description(final_df):
    table = str.maketrans("", "", string.punctuation)
    descriptions = []
    for desc, key in final_df[['caption','pixel_values']].values:
        desc = tokenizer.tokenize(desc)
        desc = [word.lower() for word in desc]
        desc = [word for word in desc if len(word)>1]
        desc =  ' '.join(desc)
        descriptions.append(desc)
    return descriptions

final_df['claned_desc'] = clean_description(final_df.copy())

In [14]:
final_df.head()

,id,caption,pixel_values,claned_desc
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...,"[[[58, 56, 77], [115, 131, 130], [106, 123, 11...",child in pink dress is climbing up set of stai...
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .,"[[[58, 56, 77], [115, 131, 130], [106, 123, 11...",girl going into wooden building
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .,"[[[58, 56, 77], [115, 131, 130], [106, 123, 11...",little girl climbing into wooden playhouse
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...,"[[[58, 56, 77], [115, 131, 130], [106, 123, 11...",little girl climbing the stairs to her playhouse
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...,"[[[58, 56, 77], [115, 131, 130], [106, 123, 11...",little girl in pink dress going into wooden cabin


In [15]:
def getVocabulary(final_df):
    all_desc = set()
    for desc, key in final_df[['claned_desc','pixel_values']].values:
        all_desc.update(tokenizer.tokenize(desc))
    return all_desc


vocabulary = getVocabulary(final_df.copy())
print('Original Vocabulary Size: %d' % len(vocabulary))

Original Vocabulary Size: 8464


In [16]:
def save_descriptions(final_df, filename):
    lines = list()
    for key, desc in final_df[['id','claned_desc']].values:
        lines.append(key + ' ' + desc)
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

save_descriptions(final_df.copy(), 'descriptions.txt')
print("Desciption.txt created")

Desciption.txt created
